In [20]:
# Check if the files exist befor
  load("/content/x_test_images.RData")
  load("/content/x_train_images.RData")
  load("/content/y_train_images.RData")


In [21]:
dim(x_train)
dim(y_train)
dim(x_test)

[1] 20000    32    32     3

[1] 20000     2

[1] 3000   32   32    3

In [22]:
# Standardise per pixel
x_train_std <- x_train / 255
dim(x_train_std)

x_test_std <- x_test / 255
dim(x_test_std)

[1] 20000    32    32     3

[1] 3000   32   32    3

In [23]:
install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("caret"):
“installation of package ‘caret’ had non-zero exit status”


In [24]:
library(caret)
set.seed(123)

trainIndex <- createDataPartition(y_train[,2], p = 0.8, list = FALSE)
dim(x_train_std)
X_train <- x_train_std[-trainIndex,,,]
X_test  <- x_train_std[-trainIndex,,,]
y_train_split <- y_train[-trainIndex,]
y_test_split  <- y_train[trainIndex,]

dim(X_train)
dim(X_test)
dim(y_train_split)
dim(y_test_split)

[1] 20000    32    32     3

[1] 3998   32   32    3

[1] 3998   32   32    3

[1] 3998    2

[1] 16002     2

In [25]:
install.packages("reticulate", dependencies = TRUE)
install.packages("keras3")
install.packages("tensorflow")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("reticulate", dependencies = TRUE):
“installation of package ‘reticulate’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("keras3"):
“installation of package ‘keras3’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("tensorflow"):
“installation of package ‘tensorflow’ had non-zero exit status”


In [26]:
library(reticulate)
use_python("/usr/bin/python3", required = TRUE)
py_config()

python:         /usr/bin/python3
libpython:      /usr/lib/python3.12/config-3.12-x86_64-linux-gnu/libpython3.12.so
pythonhome:     //usr://usr
version:        3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
numpy:          /usr/local/lib/python3.12/dist-packages/numpy
numpy_version:  2.0.2

NOTE: Python version was forced by use_python() function

In [27]:
library(keras3)
library(tensorflow)

In [28]:
# Starter model
# Define CNN model
cnn_model <- keras_model_sequential() %>%
  layer_conv_2d(
    filters = 16,
    kernel_size = 1,
    activation = 'relu',
    input_shape = c(32, 32, 3)
  ) %>%
  layer_flatten() %>%
  layer_dense(units = 128, activation = 'relu') %>%
  layer_dense(units = 10, activation = 'softmax')

In [29]:
# Mediocre model
# Define CNN model
cnn_model <- keras_model_sequential() %>%
  layer_conv_2d(
    filters = 32,
    kernel_size = 3,
    activation = 'relu',
    input_shape = c(32, 32, 3)
  ) %>%
  layer_flatten() %>%
  layer_dense(units = 128, activation = 'relu') %>%
  layer_dropout(rate = 0.5) %>%
  layer_dense(units = 10, activation = 'softmax')


In [30]:
# State of the Art Model
# Define CNN model
cnn_model <- keras_model_sequential() %>%
  layer_conv_2d(
    filters = 32,
    kernel_size = 3,
    activation = 'relu',
    input_shape = c(32, 32, 3)
  ) %>%
  layer_max_pooling_2d(pool_size = 2) %>%
  layer_conv_2d(
    filters = 64,
    kernel_size = 3,
    activation = 'relu'
  ) %>%
  layer_max_pooling_2d(pool_size = 2) %>%
  layer_dropout(rate = 0.25) %>%
  layer_flatten() %>%
  layer_dense(units = 128, activation = 'relu') %>%
  layer_dropout(rate = 0.5) %>%
  layer_dense(units = 10, activation = 'softmax')


In [31]:
# One-hot encode data
y_train_onehot <- to_categorical(y_train_split[,2], num_classes = 10)

# 2. COMPILE with sparse loss function
cnn_model %>% compile(
  optimizer = 'adam',
  loss = 'categorical_crossentropy',
  metrics = c('accuracy')
)

# 3. TRAIN MODEL
history <- cnn_model %>% fit(
  X_train, y_train_onehot,
  epochs = 20,
  batch_size = 128,
  validation_split = 0.2
)

In [ ]:
# 1. Get predicted probabilities
test_results <- cnn_model %>% predict(X_test)
test_set <- apply(test_results, 1, which.max)
accuracy <- sum(test_set == y_test_split[,2]) / length(y_train)
cat("Test accuracy:", accuracy, "\n")

In [ ]:
install.packages("tidyverse")

In [ ]:
library(tidyverse)
# PRINT FINAL CSV
final_probs <- cnn_model %>% predict(x_test_std)
final_probs <- as.data.frame(final_probs)

# Add ID column as first column
results <- final_probs %>%
  mutate(ID = seq(from = 20001, length.out = nrow(final_probs))) %>%
  select(ID, everything())

# Rename class columns dynamically
colnames(results) <- c("ID", paste0("Class", 0:(ncol(results)-2)))

write.csv(results, "CNN-submission.csv", row.names = FALSE)